# Import Libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.autograd import Variable

from torchvision import datasets, transforms
import torchvision

import sys, os
import matplotlib.pyplot as plt

import custom_resnet
from utils import CIFAR10Dataset, CIFAR10DataModule

from calc_loss_accuracy import model_training, model_testing 
from plots import plot_losses,plot_images 

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np

from lightning.pytorch.loggers import TensorBoardLogger

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.8/dist-packages/torch_lr_finder/lr_finder.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
### this is for running in local ###
try:
    os.environ['HTTP_PROXY']='http://185.46.212.90:80'
    os.environ['HTTPS_PROXY']='http://185.46.212.90:80'
    print ("proxy_exported")
except:
    None

proxy_exported


# Model Params, optimizer, loss criterion and model summary
Can't emphasize on how important viewing Model Summary is.
Unfortunately, there is no in-built model visualizer, so we have to take external help

In [3]:
import pytorch_lightning as pl

# Initialize DataModule and Model
data_module = CIFAR10DataModule()
data_module.setup('fit')


Files already downloaded and verified
Files already downloaded and verified


In [4]:
print (len(data_module.train_loader()))
print (len(data_module.val_loader()))

782
157


In [5]:
m = custom_resnet.ResNetLightningModel()

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = pl.Trainer(max_epochs=20,precision=16,logger=logger)
# Then you can train
trainer.fit(m,data_module.train_loader(), data_module.val_loader())




Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/4
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/4
Initializing distributed: GLOBAL_RANK: 2, MEMBER: 3/4
Initializing distributed: GLOBAL_RANK: 3, MEMBER: 4/4
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 4 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 3 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
LOCAL_RANK: 2 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name       | Type               | Params
--------------------------------------------------
0 | tra

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

1.59e-05
1.59e-05
1.59e-05
1.59e-05
1.630268610897928e-051.630268610897928e-051.630268610897928e-051.630268610897928e-05



1.6705372217958556e-051.6705372217958556e-051.6705372217958556e-051.6705372217958556e-05



1.7108058326937836e-051.7108058326937836e-051.7108058326937836e-051.7108058326937836e-05



1.7510744435917116e-051.7510744435917116e-051.7510744435917116e-051.7510744435917116e-05



1.7913430544896392e-051.7913430544896392e-05
1.7913430544896392e-05
1.7913430544896392e-05

1.8316116653875672e-051.8316116653875672e-051.8316116653875672e-051.8316116653875672e-05



1.8718802762854952e-051.8718802762854952e-051.8718802762854952e-051.8718802762854952e-05



1.9121488871834228e-051.9121488871834228e-051.9121488871834228e-051.9121488871834228e-05



1.9524174980813508e-051.9524174980813508e-051.9524174980813508e-051.9524174980813508e-05



1.9926861089792788e-051.9926861089792788e-051.9926861089792788e-051.9926861089792788e-05



2.0329547198772064e-052.0329547198772064e-052.03

5.2947122026093636e-055.2947122026093636e-055.2947122026093636e-05


5.2947122026093636e-05
5.334980813507291e-055.334980813507291e-055.334980813507291e-05


5.334980813507291e-05
5.375249424405219e-055.375249424405219e-055.375249424405219e-05


5.375249424405219e-05
5.415518035303147e-055.415518035303147e-05

5.415518035303147e-055.415518035303147e-05

5.455786646201075e-055.455786646201075e-055.455786646201075e-05


5.455786646201075e-05
5.496055257099003e-055.496055257099003e-055.496055257099003e-05


5.496055257099003e-05
5.536323867996931e-055.536323867996931e-055.536323867996931e-05


5.536323867996931e-05
5.5765924788948584e-055.5765924788948584e-055.5765924788948584e-05


5.5765924788948584e-05
5.616861089792786e-055.616861089792786e-055.616861089792786e-05


5.616861089792786e-05
5.6571297006907144e-055.6571297006907144e-055.6571297006907144e-05


5.6571297006907144e-05
5.697398311588642e-055.697398311588642e-055.697398311588642e-05


5.697398311588642e-05
5.73766692248657e-05

...

0.0015155030698388334
0.0015155030698388334
0.0015155030698388334
0.0015155030698388334
0.0015159057559478127
0.00151590575594781270.0015159057559478127
0.0015159057559478127

0.0015163084420567920.001516308442056792

0.001516308442056792
0.001516308442056792
0.00151671112816577140.00151671112816577140.0015167111281657714


0.0015167111281657714
0.00151711381427475050.00151711381427475050.0015171138142747505


0.0015171138142747505
0.001517516500383730.00151751650038373

0.001517516500383730.00151751650038373

0.0015179191864927092
0.00151791918649270920.0015179191864927092

0.0015179191864927092
0.00151832187260168840.00151832187260168840.0015183218726016884


0.0015183218726016884
0.00151872455871066770.0015187245587106677

0.00151872455871066770.0015187245587106677

0.0015191272448196470.0015191272448196470.001519127244819647


0.001519127244819647
0.00151952993092862640.0015195299309286264

0.0015195299309286264
0.0015195299309286264
0.00151993261703760560.00151993261703760560.0015

0.0015533555640828856
0.00155335556408288560.0015533555640828856
0.0015533555640828856

0.001553758250191865
0.0015537582501918650.001553758250191865
0.001553758250191865

0.00155416093630084430.00155416093630084430.0015541609363008443


0.0015541609363008443
0.00155456362240982350.0015545636224098235

0.00155456362240982350.0015545636224098235

0.00155496630851880280.0015549663085188028
0.0015549663085188028

0.0015549663085188028
0.00155536899462778220.0015553689946277822

0.00155536899462778220.0015553689946277822

0.00155577168073676150.00155577168073676150.0015557716807367615


0.0015557716807367615
0.00155617436684574060.0015561743668457406

0.00155617436684574060.0015561743668457406

0.00155657705295471980.0015565770529547198

0.0015565770529547198
0.0015565770529547198
0.00155697973906369910.00155697973906369910.0015569797390636991


0.0015569797390636991
0.00155738242517267850.00155738242517267850.0015573824251726785


0.0015573824251726785
0.00155778511128165760.0015577851112

0.00158959339104859350.00158959339104859350.0015895933910485935


0.0015895933910485935
0.00158945785473145780.00158945785473145780.0015894578547314578


0.0015894578547314578
0.00158932231841432230.0015893223184143223

0.00158932231841432230.0015893223184143223

0.00158918678209718670.00158918678209718670.0015891867820971867


0.0015891867820971867
0.00158905124578005120.00158905124578005120.0015890512457800512


0.0015890512457800512
0.00158891570946291570.00158891570946291570.0015889157094629157


0.0015889157094629157
0.00158878017314578020.00158878017314578020.0015887801731457802


0.0015887801731457802
0.00158864463682864450.00158864463682864450.0015886446368286445


0.0015886446368286445


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [6]:
print(trainer.callback_metrics)

{'train_loss': tensor(1.6691), 'train_loss_step': tensor(1.7303), 'train_acc': tensor(0.7928), 'train_acc_step': tensor(0.7500), 'lr': tensor(0.0016), 'lr_step': tensor(0.0016), 'val_acc': tensor(0.8410), 'train_loss_epoch': tensor(1.6691), 'train_acc_epoch': tensor(0.7928), 'lr_epoch': tensor(0.0016)}


In [7]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs

## Calculating the max and min LR using one cycle LR policy

In [8]:
#  # to reset the model and optimizer to their initial state
# learning_r_finder(m,optimizer,criterion, device, train_loader, n_iters=200,  end_lr=10)
# # sys.exit()

# Let's Train and test our model
### using one cycle lr policy

## Displaying Train Test Accuracy and Loss Plots

In [9]:
os.system('mkdir images')
fig = plot_losses(train_losses, train_acc, test_losses, test_acc)
fig.savefig('images/Accuracy & Loss.jpg')

mkdir: cannot create directory ‘images’: File exists


NameError: name 'train_losses' is not defined

## Displaying Sample Miss Classified Images

In [ ]:
# Get the misclassified data from test dataset
misclassified_data = get_misclassified_data(m, device, test_loader)
miss_classified_images = display_mis_images(misclassified_data,10, classes)
miss_classified_images.savefig("images/miss_class.jpg")
#misclassified_data.savefig("images/miss_class.jpg")

## Displaying Sample Train Dataset after trasformation

In [ ]:
batch_data_train, batch_label_train = next(iter(train_loader))
figure_train = plot_images(batch_data_train, batch_label_train.tolist(), 12, 3, 'CIFAR10')
figure_train.savefig("images/train_dataset.jpg")

## Displaying Sample Test Dataset

In [ ]:
batch_data_test, batch_label_test = next(iter(test_loader))
figure_test = plot_images(batch_data_test, batch_label_test.tolist(), 20, 4, 'CIFAR10')
figure_test.savefig("images/test_dataset.jpg")

## Grad Cam Images

In [ ]:
from utils import display_gradcam_output

# Denormalize the data using test mean and std deviation
inv_normalize = transforms.Normalize(
    mean=[-0.50/0.23, -0.50/0.23, -0.50/0.23],
    std=[1/0.23, 1/0.23, 1/0.23]
)

target_layers = [m.layer4[-1]]
targets = None

display_gradcam_output(misclassified_data, classes, inv_normalize, m, target_layers, targets, number_of_samples=20, transparency=0.80)